In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q transformers
!pip install -q llama-index
!pip install llama-index chromadb --quiet
!pip install chromadb
!pip install sentence-transformers
!pip install langchain

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pathlib import Path
from llama_index import download_loader

JSONReader = download_loader("JSONReader")

loader = JSONReader()
documents = loader.load_data(Path('/content/drive/MyDrive/instigpt_data/resobin_courses_new.json'))


In [6]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

In [7]:
import pandas as pd
import chromadb
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from IPython.display import Markdown, display

from llama_index import (VectorStoreIndex,
                         SimpleDirectoryReader,
                         ServiceContext
                         )

In [6]:
# data_folder_path = "/content/data_folder"

In [7]:
# documents = SimpleDirectoryReader(data_folder_path).load_data()

In [8]:
len(documents)

4003

Decide which model you want to use from the following list of quantized models

In [23]:
model_dictionary = {
    "open_chat_3.5": "https://huggingface.co/TheBloke/openchat_3.5-GGUF/resolve/main/openchat_3.5.Q5_K_M.gguf",
    "mistral": ["https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
                "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/blob/main/mistral-7b-instruct-v0.1.Q4_0.gguf",
                "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/blob/main/mistral-7b-instruct-v0.1.Q4_K_S.gguf",
                "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/blob/main/mistral-7b-instruct-v0.1.Q5_0.gguf",
                "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/blob/main/mistral-7b-instruct-v0.1.Q5_K_M.gguf",
                "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/blob/main/mistral-7b-instruct-v0.1.Q5_K_S.gguf",
                "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/blob/main/mistral-7b-instruct-v0.1.Q6_K.gguf",
                "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/blob/main/mistral-7b-instruct-v0.1.Q8_0.gguf"
                ],
    "zephyr_beta": ["https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/blob/main/zephyr-7b-beta.Q4_0.gguf",
                    "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/blob/main/zephyr-7b-beta.Q4_K_M.gguf",
                    "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/blob/main/zephyr-7b-beta.Q4_K_S.gguf",
                    "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/blob/main/zephyr-7b-beta.Q5_0.gguf",
                    "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/blob/main/zephyr-7b-beta.Q5_K_M.gguf",
                    "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/blob/main/zephyr-7b-beta.Q5_K_S.gguf",
                    "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/blob/main/zephyr-7b-beta.Q6_K.gguf",
                    "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/blob/main/zephyr-7b-beta.Q8_0.gguf"
                    ]
}

In [1]:
import torch
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=model_dictionary["mistral"][0],
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=8000,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [14]:
from llama_index.embeddings import HuggingFaceEmbedding

model_name="sentence-transformers/all-MiniLM-L6-v2"
embed_model = HuggingFaceEmbedding(model_name=model_name)
# embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large")

In [15]:
%pwd

'/content'

Change 'chunk_siz', 'chunk_overlap' and 'model'. Model embeddings will be saved at the given embeddings_path

In [16]:
chunk_size=128
chunk_overlap=30
model="all-MiniLM-L6-v2"
# embeddings_path="/content/drive/MyDrive/resobin_embeddings/all-MiniLM-L6-V2_128_30"
embeddings_path=f"/content/drive/MyDrive/resobin_embeddings/{model}_{chunk_size}_{chunk_overlap}"

In [18]:
# Save to disk
# DO NOT RUN THIS CODE, if you want to test with embeddings created by someone else, use the code in next cell
db = chromadb.PersistentClient(path=embeddings_path)
chroma_collection = db.get_or_create_collection("resobin_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    llm=llm,
    embed_model=embed_model,
    )

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)


In [19]:
# load from disk
# This code is used to load embeddings of a model present at embeddings_path
db2 = chromadb.PersistentClient(path=embeddings_path)
chroma_collection = db2.get_or_create_collection("resobin_collection")
service_context = ServiceContext.from_defaults(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    llm=llm,
    embed_model=embed_model,
    )

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)


In [20]:
from llama_index import get_response_synthesizer
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor

In [22]:
# Query Data from the persisted index
query_engine = index.as_query_engine(similarity_top_k=35)
response = query_engine.query("Give me a list of all the course by Energy Science and Engineering.")
display(Markdown(f"<b>{response}</b>"))


Llama.generate: prefix-match hit


<b> Based on the provided context information, here is a list of courses offered by the "Energy Science and Engineering" department:

1. EN408: Energy Design Project
2. EN653: Energy Policy Analysis
3. EN409: Mathematical Foundation for Energy Science
4. EN110: Energy Engineering Fundamentals
5. EN618: Energy Systems Modelling & Analysis
6. EN401: Energy Systems modelling and analysis
7. EN405: Energy Innovation Lab
8. EN792: Communication Skills -II
9. EN661: Micro and Nanoscale Energy Transport
10. EN628: Materials and Devices for Energy Applications
11. CM606: Energy Resources, Economics and Environment
12. EN411: Physics for Energy Science
13. EN705: An Introduction to Energy Systems Engineering
14. EN601: Nonconventional Energy Sources
15. EN607: Energy Management
16. ES616: Energy Conversion and Environment
17. EN21</b>

In [29]:
# response

Response(response=' Process Plant Simulation is a course offered by the Chemical Engineering department at the university. It covers various approaches to plant simulation, including steady-state sequential introduction to process model building and simulation; flowsheet topology representation methods and recycle detection algorithms; steady-state sequential modular simulation techniques, concepts of partitioning and tearing; and equation oriented simulation techniques. The course also covers solution methods for sparse systems and simultaneous modular simulation techniques. Additionally, it includes problem solving using selected commercial simulators and thermodynamic procedures in process simulators.', source_nodes=[NodeWithScore(node=TextNode(id_='6427c960-aece-4899-aae6-e051af493ebf', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e3908a7d-d051-4ecc-bce1-a9f978d46

In [21]:
# # configure retriever
# retriever = VectorIndexRetriever(
#     index=index,
#     similarity_top_k=10,
# )

# # configure response synthesizer
# response_synthesizer = get_response_synthesizer()

# # assemble query engine
# query_engine = RetrieverQueryEngine(
#     retriever=retriever,
#     response_synthesizer=response_synthesizer,
#     node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
# )

# # query
# response = query_engine.query("Give me the description of SC646 course.")
# display(Markdown(f"<b>{response}</b>"))
